# 네이버 지식인 크롤링


In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [6]:
driver = webdriver.Chrome('chromedriver')

In [20]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url+quote('여친선물')
driver.get(url)

In [21]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [22]:
lis = soup.select('ul.basic1>li')
len(lis)

10

In [23]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url


'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [25]:
driver.get(sub_url)

In [26]:
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
answer_page = (count-1) // 5
print(count, answer_page).


8 1


In [27]:
for i in range(answer_page):
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [32]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [34]:
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

8

In [38]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?
저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..
뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.
하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!
​
제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.
좋은 여친선물이 되셨으면 좋겠네요.


In [40]:
driver.back()

In [41]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    count = int(driver.find_element_by_css_selector('._answerCount.num').text)
    answer_page = (count - 1) // 5
    print(count, end=', ')
    for i in range(answer_page):
        driver.find_element_by_id('nextPageButton').click()
        time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.select('div.answer-content__item._contentWrap._answer')
    print(len(divs))
    for div in divs:
        ps = div.select('p.se-text-paragraph')
        for p in ps:
            text = p.get_text()
            answer_list.append(text)

    driver.back()
    time.sleep(2)



8, 8
8, 8
20, 20
8, 8
3, 3
8, 8
10, 10
10, 10
13, 13
12, 12


In [42]:
answer_list[100:110]

['http://expertreview.shop?i=277999285',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '\u200b',
 '오늘 하루도 행복한 하루되세요~! 답변채택이 번거롭더라도 지금 채택하시면 내공의 50%를 돌려받으실 수 있습니다. 채택부탁드립니다~ 제휴마케팅 일환으로 소정의 수수료를 받을 수 있습니다!',
 '\u200b',
 '\u200b']

## 1~10 페이지 데이터 한번에 하기


In [46]:
answer_list=[]
for page in range(1,11):
    print('page', page)
    url = f"{base_url}{quote('여친선물')}&page={page}"
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    lis = soup.select('ul.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count - 1) // 5
        #print(count, end=', ')
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        #print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            for p in ps:
                text = p.get_text()
                answer_list.append(text)

        driver.back()
        time.sleep(2)



page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10


In [48]:
len(answer_list)

8687

In [49]:
answer_list[0]

'혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?'

In [51]:
'''with open('new_gift.txt', 'w')
    fp.write(answer_list[:3])'''
with open('data/new_gift.txt', 'w', encoding = 'utf-8') as fp:
    for answer in answer_list:
        fp.write(answer+'\n')

In [52]:
driver.close()